In [1]:
import os, sys
sys.path.append("..")
from config.load_config import load_yaml_config, to_dict, recursive_namespace
import torch
import numpy as np
from torch import Tensor
from importlib import reload

import pytorch_lightning as pl

### Generate dataset

In [2]:
from data.synthetic.SyntheticMeshPopulation import SyntheticMeshPopulation
from data.SyntheticDataModules import SyntheticMeshesDataset
from torch.utils.data import DataLoader
from utils.helpers import get_datamodule, get_coma_args

In [13]:
config = load_yaml_config("../config_files/config_folded_c_and_s.yaml")
coma_args = get_coma_args(config, dm)
coma_args["phase_input"] = False

In [14]:
dm = get_datamodule(config)

Retrieving synthetic population from cached file.


### `Decoder3DMesh`

In [30]:
import models.Model3D as Model3D
Model3D = reload(module=Model3D)
Decoder3DMesh = Model3D.Decoder3DMesh
Encoder3DMesh = Model3D.Encoder3DMesh

In [31]:
coma_args["num_conv_filters_dec"] = coma_args["num_conv_filters_dec_c"]
dec_config = {k: v for k,v in coma_args.items() if k in Model3D.DECODER_ARGS}
enc_config = {k: v for k,v in coma_args.items() if k in Model3D.ENCODER_ARGS}
enc_config["latent_dim_content"] = dec_config["latent_dim_content"] = 45

In [32]:
decoder_3d = Decoder3DMesh(**dec_config)

In [35]:
z = Tensor(np.random.random(45)).unsqueeze(0)
decoder_3d(z)

tensor([[[ 0.1424, -0.2656, -0.9021],
         [ 0.1052, -1.1682,  0.3911],
         [ 0.9080, -0.6099, -0.8971],
         ...,
         [ 0.5765,  0.3463,  1.0953],
         [-0.1445,  1.2829,  2.2770],
         [ 0.4994,  0.7983,  1.4230]]], grad_fn=<AddBackward0>)

___

### `Encoder3DMesh`

In [ ]:
import models.Model3D as Model3D
Model3D = reload(module=Model3D)
Decoder3DMesh = Model3D.Decoder3DMesh
Encoder3DMesh = Model3D.Encoder3DMesh

In [ ]:
import models.Model4D as Model4D
import models.EncoderPLModule as EncoderPLModule

Model4D = reload(module=Model4D)
EncoderPLModule = reload(module=EncoderPLModule)
EncoderTemporalSequence = Model4D.EncoderTemporalSequence
CineComaEncoder = EncoderPLModule.CineComaEncoder

___

### `EncoderTemporalSequence`

In [ ]:
from models.Model4D import EncoderTemporalSequence
from models.EncoderPLModule import CineComaEncoder

In [ ]:
cine_encoder = EncoderTemporalSequence(enc_config, z_aggr_function="DFT", n_timeframes=20)

Let's try the model on a single datapoint:

In [ ]:
datapoint = next(iter(dm.train_dataloader()))
s_t = datapoint["s_t"]
cine_encoder(s_t)["mu"]

In [ ]:
PLEncoder = CineComaEncoder(cine_encoder, config)
trainer = pl.Trainer()

In [ ]:
trainer.fit(PLEncoder, dm)

___

### `ContentDecoder`

In [ ]:
from models.Model import ContentDecoder

___

### `StyleDecoder`

In [ ]:
from models.Model import StyleDecoder

___

### `Coma4D_C_and_S`

In [ ]:
from models.Model import Coma4D_C_and_S